One way to think of your computer's memory is as having several slots in which to store things and each slot has an address.

In [3]:
x = 2
x

2

You can check the memory address of an object in Python with the id() function.

In [5]:
?id

Signature: id(obj, /)
Docstring:
Return the identity of an object.

This is guaranteed to be unique among simultaneously existing objects.
(CPython uses the object's memory address.)
Type:      builtin_function_or_method

In [4]:
id(x)

140726716461896

### Notes from *grokking algorithms*

Every time you store an item in memory, a memory address is associated with it indicating the "slot" where that object is stored.

The Grokking's algorithm book indicates that there are two main ways to store multiple items - arrays and lists. Which data structure to use depends on the context. It is worth noting that Python has just a single object -- the list [] -- but other programming languages make a distinction between lists and arrays.

*Insertion*

With arrays, the objects in the container will be stored contiguously in memory. This means that if you have an array of three objects and you'd like to add a fourth but the fourth memory slot is taken up by some other object, your computer will have to shift allocation of memory around. This makes insertion into arrays relatively slow - O(n).

With linked lists, the items in your container can be anywhere in memory -- objects do not need to be stored "next" to one another. The first slot has data about the object in the list is stored together with the memory address of the next object in the list. Adding an item to the list does not require the computer to move items when you insert new values; you just store the address of the new item with the previous item's data. This makes insertion a new item to the end of the list very fast - O(1).

*Reading*

Reading a linked list is a different story. You cannot simply read the last element or middle element in the linked list because you don't know their addresses. If you are going to read all of the items in the list, linked lists are fine but arrays allow you to grab items at a particular index.

That is, linked lists ONLY allow **sequential access.** Arrays allow for both sequential access and *random access.* With random access, you can easily jump to the 20th element in a list without having to read the first 19. Arrays are much faster at reads.

*Deleting*

With an array, everything in the container will need to be moved up by one when you delete an object.

|               | Reading | Insertion | Deletion |
| ------------- | ------- | --------- | -------- |
| Arrays        | O(1)    | O(n)      | O(1)     |
| Linked Lists  | O(n)    | O(1)      | O(1)     |

With linked lists, deletions are only O(1) if you can instantly access the element to be deleted. It is common practice to keep track of the first and last element of the linked list so it only takes O(1) time to delete those but taking into account the aforemention structure, if you want to deleting a middle item in a linked list, you'll need to find the location within the collection before you can delete it.

## Python's List Implementation

Python implements its lists as "dynamic" arrays. Dynamic arrays are arrays with automatic resizing so you do not need to specify the number of elements your array will hold ahead of time. Python's list will automatically allocate additional memory slots for your list following the growth pattern: 0, 4, 8, 16, 25, 35, 46, 58, 72, 88, ...

http://www.laurentluce.com/posts/python-list-implementation/ - An excellent article here on how Python lists are implemented. Note that it was written **twelve years ago!** so, I am not sure how accurate is for the current version of Python. Still, it offered some really useful intuition for me so I am keeping the notes I took while reading it here. If someone else is reading this for some reason though, the blog article is going to be much better than my summary.

Anywho, sandard Python is CPython and written in C.

When you initialize an empty list in Python, C returns a list object to you. There is nothing in the list so no memory is allocated for the list's *contents*.


In [67]:
my_list = []
my_list

[]

In [68]:
my_list.append(1)
my_list.append(2)
my_list.append(3)
my_list.append(4)
my_list

[1, 2, 3, 4]

When you append an integer to the list, an internal C function called app1() is called. It resizes the list from zero to four -- putting aside four slots in memory for your list objects though you've only appended one. It does this to avoid having to resize the list every time you append an item.

When you append three more items (index 1, 2, 3), there is no need to allocate more memory. That memory has already been allocated.

It isn't until you hit the 5th item (index 4), that more memory will need to allocated - four more slots are allocated for a total of eight memory slots. (Again, growth pattern is 0, 4, 8, 16, 25, 35, 46, 58, 72, 88, ...)

In [69]:
my_list.insert(1, 5)
my_list

[1, 5, 2, 3, 4]

**Insertions**

If you want to inset an integer (5) in the middle of the list at position 1, four more slots of memory will be allocated/the list will be resized. For our list of size 5, there will be 8 memory slots allocated.

Additionally, the elements at index 1, 2 and 3 will be shifted to indices 2, 3 and 4. This explains why insertion is O(n)

**Pop**

In [70]:
my_list.pop()

4

In [71]:
my_list

[1, 5, 2, 3]

When you pop the last element, C's listpop() function is called.

The size of the list changes to 4. The 4th slot in memory still points to the integer 4 but it is returned as the return element for the pop function.

If the list is empty, null is returned.

The list is resized IF the new size is strictly less than half the allocated size. So, above, our allocated size is 8 memory slots. We we have popped the fifth element so my_list's "new size" is 4. This is not **less than** half the allocated memory so no change is made.

If we pop again:



In [72]:
my_list.pop()

3

In [73]:
my_list

[1, 5, 2]

Our size is 3, less than half the allocated memory slots so my_list will be resized. 

Slots 3 and 4 will still point to some integers with slot 3's value being returned as our element to pop but the new size of the list is now 3.

There are 6 memory slots allocated.*

Our list of size 3 is associated with four memory slots and there are two additional memory slots associated with the popped values.

**Remove**

In [74]:
my_list

[1, 5, 2]

In [75]:
my_list.remove(5)

In [76]:
my_list

[1, 2]

When we call the remove method on a list object, C's listremove() is called.

listremove loops through each list element (O(n)) and, when it hits the correct element, it will use a function called list_ass_slice() to remove the element. (hehe, ass slice.)

Prior to removal, we can find 5 at index 1 so the "low offset" for list_ass_slice is index 1. The high offset is 2 (low offset + 1).

The integer 5 is "recycled to dereference it."
Elements are shifted from slot 2 (high offset) to slot 1 (low offset).

The list's size is 2 and there are 5 memory slots allocated.

The memory slot at 0 still points to the integer 1.
The memory slot at 1 *was* pointing to the integer 5 but now points to the integer 2 (shifted).
The memory slot at 2 is STILL pointing to the integer 2 but the SIZE of the list is only two now (0 and 1 are the only ones "really" allocated".)
The memory slot at 3 and 4 point to the popped integers from before.

The integer objects that are "dangling" (is that the right word to use?) will be cleaned up by the garbage collector when it sees there are no more references to these objects.


**Python List Operations - Time Complexity**

https://wiki.python.org/moin/TimeComplexity

| Operation    | Average Case | Amortized Worst Case |
| ------------ | ------------ | -------------------- |
| append[1]    | O(1)         | O(1)                 |
| pop (last)   | O(1)         | O(1)                 |
| pop (interm) | O(n)         | O(n)                 |
| insert       | O(n)         | O(n)                 |
| delete       | O(n)         | O(n)                 |

### Notes from Real Python + Minor Embellishments

https://realpython.com/linked-lists-python/

Linked lists are ordered collections.

This article goes onto further formalize each element of a linked list as being a "node" with each node containing 1) data and 2) "next" - a reference to the next node on the list.

The first node is called the "head" of the linked list. It is used as the starting point for any iteration.

Linked lists are used to implement queues, stacks and graphs.

**Queues and Stacks**

Queues and stacks differ in the way elements are retrieved.

For a queue, you use a first-in/first-out approach. That is, the first element inserted in the list is the first one to be retrieved. With a queue data structure, elements will be continuously inserted and removed at the beginning of the list.

When you append new elements to the queue (or new people stand in line), they go the end of the line.

Stacks are Last-In/First-Out. The last element inserted in the list is the first to be retrieved. You're making a stack of pancakes and you place it on the table. The last pancake made is the "top" pancake and it is eaten before the other pancakes.

**Graphs**

Linked lists can also be used to represent graphs (think network analysis) using an adjacency list (which is a list of linked lists). But, graphs are not really a focus of this particular article so I won't elaborate on that just yet.



**Performance Comparison: Lists vs Linked Lists**

In other languages, linked lists and arrays are stored in memory very differently but, as indicated above, Python programmers mainly use lists which are dynamic arrays. The main focus with Python when it comes to performance differences between lists and linked lists is *time complexity.*

As we saw, when you insert or remove and element **anywhere other than the end** of a classic Python list, there is some shifting that has to occur in the background -- resulting in O(n) time complexity. When you insert at the beginning of the Python list, all of the elements in the list have to be shifted in memory O(n).

Recall that linked lists keep track of the head node and the end node so insertion and deletion at the beginning of the list and the end of the list will **both** be O(1) - constant time complexity.

*Implication for queue: Use a linked list*

The implication of this is that linked lists have a performance advantage when you are implementing a queue -- elements are continuously inserted and removed at the beginning of the list. 

*Implication for stack: Use either a linked list or a list*

Linked lists will perform similarly to the dynamic array/Python's list structure when implementing a stack data structure where elements are being inserted/removed at the end of the list.

However, recall that arrays allow **random access** and the linked list will only allow you to perform sequential access. If you know what element you want to access, you can get the element you want in O(1) time from an array because there is no need to traverse the entire list potentially looking for it.

**collections.deque**

https://docs.python.org/3.7/library/collections.html#collections.deque

Python's collections module contains a function called deque ("deck") that returns a "deck" object. It is a generalization of stacks and queues and will append and pop from the end of the beginning with approximately O(1) performance. They do not incur the O(n) memory movement costs pop(0) and insert(0, v) do.

In [78]:
from collections import deque
d = deque()
d

deque([])

In [79]:
type(d)

collections.deque

In [80]:
?deque

Init signature: deque(self, /, *args, **kwargs)
Docstring:     
deque([iterable[, maxlen]]) --> deque object

A list-like sequence optimized for data accesses near its endpoints.
File:           c:\users\shea\appdata\local\programs\python\python311\lib\collections\__init__.py
Type:           type
Subclasses:     

In [83]:
# An empty deque object is created if no argument is passed
# deque() can also take an iterable (lists, tuples, dicts, strings)

deque('abc')

deque(['a', 'b', 'c'])

In [87]:
linked_list = deque("abcde")
linked_list

deque(['a', 'b', 'c', 'd', 'e'])

In [88]:
# append to end
linked_list.append("f")
linked_list

deque(['a', 'b', 'c', 'd', 'e', 'f'])

In [89]:
# pop from end
linked_list.pop()

'f'

In [90]:
linked_list

deque(['a', 'b', 'c', 'd', 'e'])

In [91]:
# append to beginning!
linked_list.appendleft("z")

In [92]:
linked_list

deque(['z', 'a', 'b', 'c', 'd', 'e'])

In [93]:
#remove from beginning
linked_list.popleft()

'z'

In [94]:
linked_list

deque(['a', 'b', 'c', 'd', 'e'])

**TODO, next up: work through "How to Implement Queues and Stacks"**

https://realpython.com/linked-lists-python/#how-to-implement-queues-and-stacks

## Main Points:

* Use a linked list when you need to implement a queue/you have elements being inserted/removed at the beginning of the list. The intuition is that the linked list keeps track of the head node as well as the end node, making insertion and deletion at the beginning of the list O(1) instead of O(n) as it is with an array.

* Python lists are implemented as dynamic arrays. Classic arrays often require you to determine ahead of time how much space you will need for an array ahead of time because the objects in the array are stored contiguously; adding additional data to a classic array means finding space in memory where the objects can be stored "next" to each other. Dynamic arrays, like Python lists, are implemented in such a way that the memory allocated is increased/decreased for you as you interact with it. Laurent Luce's blog post in Resources explains well what is happening behind the scenes in C.

* Python offers the "deque" object, returned from the deque() function within the standard library's "collection" module. The deque() function takes an iterable and returns a collection much like a linked list. (I'm not sure exactly how it is implemented.) It has ~O(1) time for insertion and removal from the beginning and end of the collection like a linked list though. Methods: append("a"), pop() for insertion and removal at end of the collection, appendleft("a"), popleft() for insertion and removal at the head/beginning of the collection.

### Questions So Far:

1. If Python lists are actually arrays, why is it that these memory addresses are not exactly sequential?

In [95]:
x = [1, 2, 3, 4]

id(x[0]), id(x[1]), id(x[2]), id(x[3])

(140726716461864, 140726716461896, 140726716461928, 140726716461960)

In [97]:
y = [12_341, "kfjdksdjf", "kittens"]

id(y[0]), id(y[1]), id(y[2])

(1941205522032, 1941208495408, 1941233512240)

### Resources:
* [grokking algorithms](https://bookshop.org/p/books/grokking-algorithms-an-illustrated-guide-for-programmers-and-other-curious-people-aditya-bhargava/10667477)
* [Linked Lists in Python](https://realpython.com/linked-lists-python/)
* [Python Docs: Time Complexity](https://wiki.python.org/moin/TimeComplexity)
* [Laurent Luce's Blog, Python List Implementation](http://www.laurentluce.com/posts/python-list-implementation/)